In [1]:
import torch
import torch.utils.data
import torchvision
import traceback

# Import the train, validation, test dataloader generator

Note that the test percentage is implicitly calculated as

     1 - train_percentage - test_percentage

In [3]:
from cocoDataloader import create_train_validation_test_loader

In [4]:
train_dataloader, valid_dataloader, test_dataloader = create_train_validation_test_loader( 
                                                        "./frames/",
                                                        "./merged_coco_annotation/merged_coco.json", 
                                                        4,
                                                        torchvision.transforms.Compose([torchvision.transforms.ToTensor()]),
                                                        "./video_file_id_map.json",                        
                                                        train_validation_test_split=(0.6, 0.2))

loading annotations into memory...
Done (t=0.35s)
creating index...
index created!
loading annotations into memory...
Done (t=0.37s)
creating index...
index created!
loading annotations into memory...
Done (t=0.30s)
creating index...
index created!
loading annotations into memory...
Done (t=0.33s)
creating index...
index created!


# Example of using the dataloader to train the model

The training code below is from: https://medium.com/fullstackai/how-to-train-an-object-detector-with-your-own-coco-dataset-in-pytorch-319e7090da5

The example essentially used a pre-trained Faster R-CNN with ResNet50 as the backbone. 

Due to the time constraint, we only trained the model for 2 epochs, but the number of epochs to train is easily adjustable. 

In [25]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model
    

# 2 classes; Only target class or background
num_classes = 2
num_epochs = 2
model = get_model_instance_segmentation(num_classes)

# move model to the right device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
model.to(device)
    
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

len_dataloader = len(train_dataloader)

for epoch in range(num_epochs):
    print(f"++++++++++++++++ epoch = {epoch} ++++++++++++++++")
    model.train()
    i = 0    
    for imgs, annotations in train_dataloader:
        i += 1
        print(f"Starting iteration: {i}")
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        try:
          loss_dict = model(imgs, annotations)
          losses = sum(loss for loss in loss_dict.values())
      
          optimizer.zero_grad()
          losses.backward()
          optimizer.step()
        except:
          # print out the images and the annotations that caused error in training
          print(imgs)
          print(annotations)
          
          trace_error = traceback.format_exc()
            
          print(trace_error)
          print()

        print(f'Iteration: {i}/{len_dataloader}, Loss: {losses}')

Streaming output truncated to the last 5000 lines.
Iteration: 209/1291, Loss: 0.3034365773200989
Starting iteration: 210
Iteration: 210/1291, Loss: 0.5336348414421082
Starting iteration: 211
Iteration: 211/1291, Loss: 0.3787373900413513
Starting iteration: 212
Iteration: 212/1291, Loss: 0.29227957129478455
Starting iteration: 213
Iteration: 213/1291, Loss: 0.42735564708709717
Starting iteration: 214
Iteration: 214/1291, Loss: 0.45282888412475586
Starting iteration: 215
Iteration: 215/1291, Loss: 0.4746309816837311
Starting iteration: 216
Iteration: 216/1291, Loss: 0.3776613175868988
Starting iteration: 217
Iteration: 217/1291, Loss: 0.28696468472480774
Starting iteration: 218
Iteration: 218/1291, Loss: 0.34899580478668213
Starting iteration: 219
Iteration: 219/1291, Loss: 0.3568064570426941
Starting iteration: 220
Iteration: 220/1291, Loss: 0.4630431532859802
Starting iteration: 221
Iteration: 221/1291, Loss: 0.38395705819129944
Starting iteration: 222
Iteration: 222/1291, Loss: 0.2701

# Example of saving the trained model and the optimizer

In [26]:
torch.save(model.state_dict(), './model_saved.pth')

In [27]:
torch.save(optimizer.state_dict(), './opt_saved.pth')